In [2]:
import pandas as pd
import plotly.express as px
import json

In [3]:
dataset = pd.read_csv("taxi_partial.csv")

In [4]:
dataset

,TRIP_ID,CALL_TYPE,ORIGIN_CALL,ORIGIN_STAND,TAXI_ID,TIMESTAMP,DAY_TYPE,MISSING_DATA,POLYLINE
0,T1,B,NaN,15.0,20000542,1408039037,A,False,"[[-8.585676, 41.148522], [-8.585712000000001, ..."
1,T2,B,NaN,57.0,20000108,1408038611,A,False,"[[-8.610876000000001, 41.14557], [-8.610858, 4..."
2,T3,B,NaN,15.0,20000370,1408038568,A,False,"[[-8.585739, 41.148558], [-8.585730000000002, ..."
3,T4,B,NaN,53.0,20000492,1408039090,A,False,"[[-8.613963, 41.141169], [-8.614125000000001, ..."
4,T5,B,NaN,18.0,20000621,1408039177,A,False,"[[-8.619902999999999, 41.148036000000005], [-8..."
...,...,...,...,...,...,...,...,...,...
315,T323,A,70885.0,NaN,20000430,1419171485,A,False,"[[-8.570196, 41.159484], [-8.570187, 41.158962..."
316,T324,B,NaN,53.0,20000020,1419170802,A,False,"[[-8.613873, 41.14123200000001], [-8.613882, 4..."
317,T325,C,NaN,NaN,20000207,1419172121,A,False,"[[-8.648100000000001, 41.152536], [-8.647461, ..."
318,T326,A,76232.0,NaN,20000667,1419171980,A,False,"[[-8.571699, 41.156073000000006], [-8.57058300..."


In [5]:
dataset.columns

Index(['TRIP_ID', 'CALL_TYPE', 'ORIGIN_CALL', 'ORIGIN_STAND', 'TAXI_ID',
       'TIMESTAMP', 'DAY_TYPE', 'MISSING_DATA', 'POLYLINE'],
      dtype='object')

In [6]:
lst = json.loads(dataset["POLYLINE"].iloc[0])
lon, lat = zip(*lst)
polyline = pd.DataFrame()
polyline["lat"] = lat
polyline["lon"] = lon
polyline["id"] = dataset["TRIP_ID"].iloc[0]
polyline

,lat,lon,id
0,41.148522,-8.585676,T1
1,41.148639,-8.585712,T1
2,41.148855,-8.585685,T1
3,41.148927,-8.585730,T1
4,41.148963,-8.585982,T1
5,41.148954,-8.586396,T1
6,41.148720,-8.586072,T1
7,41.147847,-8.586324,T1
8,41.147460,-8.586999,T1
9,41.147154,-8.586576,T1


In [7]:
df = dataset.loc[:, ["POLYLINE", "TRIP_ID"]]
polylines = pd.DataFrame()
lon, lat, id = [], [], []

for i in range(len(df.index)):
    try:
        lst = json.loads(dataset["POLYLINE"].iloc[i])
        longi, latte = zip(*lst)
        lon = lon+ list(longi)
        lat=lat+list(latte)
        id = id+[dataset["TRIP_ID"].iloc[i] for j in range(len(lst))]
    except:
        print("upsie doopsie")
    
polylines["lat"]=lat
polylines["lon"]=lon
polylines["id"]=id

polylines

,lat,lon,id
0,41.148522,-8.585676,T1
1,41.148639,-8.585712,T1
2,41.148855,-8.585685,T1
3,41.148927,-8.585730,T1
4,41.148963,-8.585982,T1
...,...,...,...
14433,41.199111,-8.582562,T327
14434,41.198310,-8.586135,T327
14435,41.197824,-8.590464,T327
14436,41.195223,-8.590779,T327


In [15]:
fig = px.density_mapbox(polylines, lat="lat", lon="lon", radius=5, zoom=3, height=700)

fig.update_layout(mapbox_style="stamen-terrain", mapbox_zoom=12, mapbox_center_lat = 41.14,
    margin={"r":0,"t":0,"l":0,"b":0})

fig.show()

In [11]:
fig = px.line_mapbox(polylines, lat="lat", lon="lon", color="id", zoom=3, height=700)

fig.update_layout(mapbox_style="stamen-terrain", mapbox_zoom=12, mapbox_center_lat = 41.14,
    margin={"r":0,"t":0,"l":0,"b":0})

fig.show()

In [32]:
from sklearn.cluster import DBSCAN
dbscan=DBSCAN(eps=0.0005,min_samples=10)
dbscan.fit(polylines[['lat','lon']])

polylines['DBSCAN_labels']=dbscan.labels_
polylines_filtered = polylines[polylines.DBSCAN_labels>-1]


# Plotting resulting clusters
#colors=['purple','red','blue','green']
#plt.figure(figsize=(10,10))
#plt.scatter(df[0],df[1],c=df['KMeans_labels'],cmap=matplotlib.colors.ListedColormap(colors),s=15)
#plt.title('K-Means Clustering',fontsize=20)
#plt.xlabel('Feature 1',fontsize=14)
#plt.ylabel('Feature 2',fontsize=14)
#plt.show()

fig = px.scatter_mapbox(polylines_filtered, lat="lat", lon="lon", zoom=3, color="DBSCAN_labels", height=700)

fig.update_layout(mapbox_style="stamen-terrain", mapbox_zoom=12, mapbox_center_lat = 41.14,
    margin={"r":0,"t":0,"l":0,"b":0})

fig.show()

In [ ]:
def align_tracks(track1, track2, gap_penalty):
    """ Needleman-Wunsch algorithm adapted for gps tracks. """
    
    _log.info("Aligning tracks")

    def similarity(p1, p2):
        d = gpxpy.geo.distance(p1.latitude, p1.longitude, p1.elevation,
                               p2.latitude, p2.longitude, p2.elevation)
        return -d

    # construct f-matrix
    f = numpy.zeros((len(track1), len(track2)))
    for i in range(0, len(track1)):
        f[i][0] = gap_penalty * i
    for j in range(0, len(track2)):
        f[0][j] = gap_penalty * j
    for i in range(1, len(track1)):
        t1 = track1[i]
        for j in range(1, len(track2)):
            t2 = track2[j]
            match = f[i-1][j-1] + similarity(t1, t2)
            delete = f[i-1][j] + gap_penalty
            insert = f[i][j-1] + gap_penalty
            f[i, j] = max(match, max(delete, insert))

    # backtrack to create alignment
    a1 = []
    a2 = []
    i = len(track1) - 1
    j = len(track2) - 1
    while i > 0 or j > 0:
        if i > 0 and j > 0 and \
           f[i, j] == f[i-1][j-1] + similarity(track1[i], track2[j]):
            a1.insert(0, track1[i])
            a2.insert(0, track2[j])
            i -= 1
            j -= 1
        elif i > 0 and f[i][j] == f[i-1][j] + gap_penalty:
            a1.insert(0, track1[i])
            a2.insert(0, None)
            i -= 1
        elif j > 0 and f[i][j] == f[i][j-1] + gap_penalty:
            a1.insert(0, None)
            a2.insert(0, track2[j])
            j -= 1
    return a1, a2

align_tracks()